In [1]:
# third party
import os
import pickle
import numpy as np
from deep_teaching_commons.data.fundamentals.mnist import Mnist

In [2]:
def loadMNISTData():
    # create mnist loader from deep_teaching_commons
    mnist_loader = Mnist(data_dir='data')

    # load all data, labels are one-hot-encoded, images are flatten and pixel squashed between [0,1]
    train_images, train_labels, test_images, test_labels = mnist_loader.get_all_data(flatten=False, one_hot_enc=False, normalized=True)
    print(train_images.shape, train_labels.shape)

    # reshape to match generel framework architecture 
    train_images, test_images = train_images.reshape(60000, 1, 28, 28), test_images.reshape(10000, 1, 28, 28)            
    print(train_images.shape, train_labels.shape)

    # shuffle training data
    shuffle_index = np.random.permutation(60000)
    train_images, train_labels = train_images[shuffle_index], train_labels[shuffle_index]
    return (train_images, train_labels, test_images, test_labels)

In [3]:
def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:      
        dict = pickle.load(f, encoding='bytes')
        X = dict[b'data']
        Y = dict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float64")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(root_folder):
    xs = []
    ys = []
    for batch in range(1,6):
        f = os.path.join(root_folder, 'data_batch_%d' % (batch, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(root_folder, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [4]:
# (train_images, train_labels, test_images, test_labels) = loadMNISTData()
(train_images, train_labels, test_images, test_labels) = load_CIFAR10('cifar-10-batches-py')

In [5]:
print('Training data shape: ', train_images.shape)
print('Training labels shape: ', train_labels.shape)
print('Test data shape: ', test_images.shape)
print('Test labels shape: ', test_labels.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [8]:
# framework modules
from htw_nn_framework.networks import *
from htw_nn_framework.activation_func import *
from htw_nn_framework.loss_func import *
from htw_nn_framework.layer import *
from htw_nn_framework.optimizer import *

# design a three hidden layer architecture with Dense-Layer
# and ReLU as activation function
def fcn_mnist():
    flat = Flatten()
    hidden_01 = FullyConnected(3072, 2000)
    relu_01 = ReLU()
    hidden_02 = FullyConnected(2000, 200)
    relu_02 = ReLU()
    hidden_03 = FullyConnected(200, 100)
    relu_03 = ReLU()
    ouput = FullyConnected(100, 10)
    return [flat, hidden_01, relu_01, hidden_02, relu_02, hidden_03, relu_03, ouput]

# create a neural network on specified architecture with softmax as score function
fcn = NeuralNetwork(fcn_mnist(), score_func=LossCriteria.softmax)

# optimize the network and a softmax loss
fcn = Optimizer.sgd(fcn, train_images, train_labels, LossCriteria.cross_entropy_softmax, batch_size=64, epoch=10, learning_rate=0.01, X_test=test_images, y_test=test_labels, verbose=True)

Epoch 1
-log( [1.05311358e-055 1.80332227e-044 4.59966920e-080 1.72939455e-028
 2.41937229e-015 1.00000000e+000 1.96732476e-006 2.97752340e-060
 3.63900596e-156 1.34338722e-091 3.24500693e-011 6.65613636e-058
 1.15440390e-130 1.00000000e+000 2.02216612e-093 5.25553867e-040
 2.61936496e-136 2.80964403e-158 7.33140966e-019 1.37208057e-017
 4.59509257e-002 5.65996943e-118 2.06669058e-012 3.24204323e-025
 6.53351560e-004 1.43345361e-068 1.00688378e-117 2.14643454e-059
 4.45272838e-092 6.61736260e-080 1.09727485e-052 4.88404068e-020
 7.06843432e-001 1.45483342e-189 1.00000000e+000 2.10461290e-072
 4.45811213e-262 5.49372877e-075 2.31166918e-196 9.77269408e-195
 7.05578544e-080 1.00000000e+000 1.00000000e+000 8.12108684e-089
 9.86757183e-001 7.73052335e-030 4.07680315e-037 9.64286178e-001
 1.00000000e+000 4.06204621e-096 3.69255470e-063 1.96772787e-053
 5.41695459e-072 1.16333536e-062 1.67305749e-011 1.61563859e-035
 4.64276505e-070 5.24357494e-047 1.69315068e-012 1.62948888e-180
 1.00000000

/Users/marpme/Desktop/uebung_3/htw_nn_framework/loss_func.py:23: RuntimeWarning: divide by zero encountered in log
  log_likelihood = -np.log(p[range(m), y])


-log( [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] )
-log( [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] )
-log( [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] )
-log( [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan] )


/Users/marpme/Desktop/uebung_3/htw_nn_framework/loss_func.py:12: RuntimeWarning: invalid value encountered in subtract
  exp_X = np.exp(X - np.max(X, axis=1, keepdims=True))
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/Users/marpme/Desktop/uebung_3/htw_nn_framework/activation_func.py:29: RuntimeWarning: invalid value encountered in less_equal
  dX[self.X <= 0] = 0


-log( [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan] )
-log( [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan] )
-log( [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan] )
-log( [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan 

KeyboardInterrupt: 